In [120]:
from sympy import exp, sin, cos, acos, tan, diffgeom, latex, pi, symbols, Rational, simplify, Function, Matrix, sqrt
from sympy import diff, Derivative
import numpy as np
import matplotlib.pyplot as plt
from sympy.diffgeom import Manifold, Patch, CoordSystem
from sympy.diffgeom import metric_to_Riemann_components as Riemann
from sympy.diffgeom import metric_to_Ricci_components as Ricci
from sympy.diffgeom import metric_to_Christoffel_2nd as Christoffel
from IPython.display import display, Math
from sympy.tensor.array import MutableDenseNDimArray as mArray

In [121]:
def print_nonzero_christoffels(metric, component_list):
# given a metrix and a list of coordinates in the form ['t', 'x', 'y', 'z']
# prints out all of the non-zero Christoffel symbols in the form Gamma^c1_c2, c3
    print('********************** Christoffel Symbols **********************')
    christoffels = Christoffel(metric)
    size = len(component_list)
    for i in range(1):
        for j in range(size):
            for k in range(size):
                simplified = simplify(christoffels[i, j, k])
                if simplified != 0:
                    print(component_list[i] + ', ' + component_list[j] + ', ' + component_list[k] + ':   ')
                    display(Math(latex(simplified)))
                    print('---------------------------------')
    print('*****************************************************************')
    return;

def print_nonzero_riemann(R, component_list):
# R is the riemann tensor
# component list is the list of indices of R in the form ['t', 'x', 'y', 'z']
# prints out R^c1_c2,c3,c4
    print('************************* Riemann Tensor *************************')
    n = np.shape(R)[0]
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for q in range(n):
                    simplified = simplify(R[i, j, k, q])
                    if simplified != 0:
                        print(component_list[i] + ', ' + component_list[j] + ', ' + component_list[k] + ', '
                              + component_list[q] + ':   ')
                        display(Math(latex(simplify(simplified))))
                        print('---------------------------------')
    print('******************************************************************')
    return;
                        
def print_nonzero_ricci(R, component_list):
# R is the Ricci tensor
# component list is the list of coordinates in the form ['t', 'x', 'y', 'z']
# prints out R_c1, c2
    print('************************* Ricci Tensor *************************')
    n = np.shape(R)[0]
    for i in range(n):
        for j in range(n):
            simplified = simplify(R[i, j])
            if simplified != 0:
                print(component_list[i] + ', ' + component_list[j])
                display(simplified)
                print('---------------------------------')
    print('******************************************************************')
    return;

def print_ricci_scalar(g):
# finds and prints the Ricci scalar
    R = Ricci(g)
    metric_matrix = diffgeom.twoform_to_matrix(g)
    metric_inv = metric_matrix.inv()
    ricci_scalar = 0
    for i in range(np.shape(R)[0]):
        for j in range(np.shape(R)[0]):
            ricci_scalar += metric_inv[i, j] * R[i, j]
    print('************************** Ricci Scalar **************************')
    display(Math(latex(simplify(ricci_scalar))))
    print('******************************************************************')
    return;

def get_ricci_scalar(g):
# finds and prints the Ricci scalar
    R = Ricci(g)
    metric_matrix = diffgeom.twoform_to_matrix(g)
    metric_inv = metric_matrix.inv()
    ricci_scalar = 0
    for i in range(np.shape(R)[0]):
        for j in range(np.shape(R)[0]):
            ricci_scalar += metric_inv[i, j] * R[i, j]

    return ricci_scalar;

def print_nonzero_einstein(metric, component_list):
    ricci_tensor = Ricci(metric)
    ricci_scalar = get_ricci_scalar(metric)
    metric_matrix = diffgeom.twoform_to_matrix(metric)
    term1 = ricci_tensor 
    term2 = - Rational(1 / 2) * metric * ricci_scalar
    #einstein = term1 + term2
    display(Math(latex(simplify(term1))))
    display(Math(latex(simplify(term2))))
    #print('************************* Einstein Tensor *************************')
    #for i in range(np.shape(einstein_tensor)[0]):
    #    for j in range(np.shape(einstein_tensor[1])):
    #        
    #print('*******************************************************************')

In [125]:
from sympy import *
components = ['t', 'r', 'theta', 'phi']
t, r, theta, phi = symbols('t, r, theta, phi')
m = Manifold('M', 4)
patch = Patch('P', m)
coords = CoordSystem('coords', patch, components)
t, r, theta, phi = coords.coord_functions()
dt, dr, dtheta, dphi = coords.base_oneforms()

f = Function('f')(r)
a = Function('a')(t)
k = Symbol('k')

# define a metric using tensor products
TP = diffgeom.TensorProduct

## function k < 0: (sqrt(6 / (k * a**2)) * sin(sqrt(k / 6) * r))
## function k = 0: (sqrt(-6 / (k * a**2)) * sinh(sqrt(-k * a**2 / 6) * r))
## function k > 0: (r)

#metric  = TP(dr, dr) + (sqrt(6 / (k)) * sin(sqrt(k / 6) * r))**2 * (TP(dtheta, dtheta)
#                        + sin(theta)**2 * TP(dphi, dphi))

metric = -TP(dt, dt) + a**2 * ((1 / (1 - k * r**2)) * TP(dr, dr) + r**2 * TP(dtheta, dtheta) + r**2 * sin(theta)**2 * TP(dphi, dphi))

print('***************************** Metric *****************************')
display(simplify(diffgeom.twoform_to_matrix(metric)))
print_ricci_scalar(metric)

***************************** Metric *****************************


Matrix([
[-1,                     0,            0,                          0],
[ 0, -a(t)**2/(r**2*k - 1),            0,                          0],
[ 0,                     0, r**2*a(t)**2,                          0],
[ 0,                     0,            0, r**2*a(t)**2*sin(theta)**2]])

************************** Ricci Scalar **************************


<IPython.core.display.Math object>

******************************************************************


In [136]:
ricci = mArray(Ricci(metric))
ricci_scalar = get_ricci_scalar(metric)
metric_matrix = diffgeom.twoform_to_matrix(metric)
metric_inv = metric_matrix.inv()

metric_matrix = mArray(metric_matrix)
metric_inv = mArray(metric_inv)

In [152]:
G = ricci - Rational(1 / 2) * metric_matrix * ricci_scalar
display(simplify(G))
print(latex(simplify(G)))

[[3*(k + Subs(Derivative(a(_xi), _xi), _xi, t)**2)/a(t)**2, 0, 0, 0], [0, (k + 2*a(t)*Subs(Derivative(a(_xi), (_xi, 2)), _xi, t) + Subs(Derivative(a(_xi), _xi), _xi, t)**2)/(r**2*k - 1), 0, 0], [0, 0, -r**2*(k + 2*a(t)*Subs(Derivative(a(_xi), (_xi, 2)), _xi, t) + Subs(Derivative(a(_xi), _xi), _xi, t)**2), 0], [0, 0, 0, -r**2*(k + 2*a(t)*Subs(Derivative(a(_xi), (_xi, 2)), _xi, t) + Subs(Derivative(a(_xi), _xi), _xi, t)**2)*sin(theta)**2]]

\left[\begin{matrix}\frac{3 \left(k + \left. \frac{d}{d \xi} a{\left(\xi \right)} \right|_{\substack{ \xi=\mathbf{t} }}^{2}\right)}{a^{2}{\left(\mathbf{t} \right)}} & 0 & 0 & 0\\0 & \frac{k + 2 a{\left(\mathbf{t} \right)} \left. \frac{d^{2}}{d \xi^{2}} a{\left(\xi \right)} \right|_{\substack{ \xi=\mathbf{t} }} + \left. \frac{d}{d \xi} a{\left(\xi \right)} \right|_{\substack{ \xi=\mathbf{t} }}^{2}}{\mathbf{r}^{2} k - 1} & 0 & 0\\0 & 0 & - \mathbf{r}^{2} \left(k + 2 a{\left(\mathbf{t} \right)} \left. \frac{d^{2}}{d \xi^{2}} a{\left(\xi \right)} \right|_{\substack{ \xi=\mathbf{t} }} + \left. \frac{d}{d \xi} a{\left(\xi \right)} \right|_{\substack{ \xi=\mathbf{t} }}^{2}\right) & 0\\0 & 0 & 0 & - \mathbf{r}^{2} \left(k + 2 a{\left(\mathbf{t} \right)} \left. \frac{d^{2}}{d \xi^{2}} a{\left(\xi \right)} \right|_{\substack{ \xi=\mathbf{t} }} + \left. \frac{d}{d \xi} a{\left(\xi \right)} \right|_{\substack{ \xi=\mathbf{t} }}^{2}\right) \sin^{2}{\left(\mathbf{\theta} \right)}\end{matrix}\right]


In [98]:
f = Function('f')
r = Symbol('r')
k = Symbol('k')
eq = -4 * Derivative(f(r), r, r) / f(r) - 2 * Derivative(f(r), r)**2 / f(r)**2 + 2 / (a**2 * f(r)**2) - k

In [101]:
solution = (sqrt(6 / (k * a**2)) * sin(sqrt(k / 6) * r))

In [148]:
rho, p = symbols('rho, p')
T = Matrix([[-rho, 0, 0, 0],
            [0, p, 0, 0],
            [0, 0, p, 0],
            [0, 0, 0, p]])

In [154]:
Matrix(T) * diffgeom.twoform_to_matrix(metric)

Matrix([
[rho,                       0,              0,                            0],
[  0, p*a(t)**2/(-r**2*k + 1),              0,                            0],
[  0,                       0, r**2*p*a(t)**2,                            0],
[  0,                       0,              0, r**2*p*a(t)**2*sin(theta)**2]])